### Knowledge Graph From Text Using llama-3.1-8b-instant

In [8]:
import os
from langchain_groq import ChatGroq
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

In [9]:
# Set your Groq API key
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
print(GROQ_API_KEY)

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


In [10]:
class Entity(BaseModel):
    name: str = Field(description="Name of the entity")
    type: str = Field(description="Type of the entity (e.g., PersonalityTrait, Ability, Interest, Skill, Knowledge etc.)")


In [11]:
class Relationship(BaseModel):
    subject: str = Field(description="Subject entity of the relationship")
    predicate: str = Field(description="Type of relationship")
    object: str = Field(description="Object entity of the relationship")

In [12]:
class KnowledgeGraphOutput(BaseModel):
    entities: List[Entity] = Field(description="List of entities extracted from the text")
    relationships: List[Relationship] = Field(description="List of relationships extracted from the text")

In [13]:
# Create the output parser
output_parser = PydanticOutputParser(pydantic_object=KnowledgeGraphOutput)

In [14]:
# Define the prompt template
prompt_template = PromptTemplate(
    template="""
    Extract entities and relationships from the following text to construct a knowledge graph:

    {text}

    {format_instructions}

    Provide the output in the specified JSON format.
    """,
    input_variables=["text"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
)

In [15]:
# Initialize the Groq LLM
llm = ChatGroq(temperature=0, model_name="llama-3.1-8b-instant")

In [16]:
# Create the LLMChain
chain = LLMChain(llm=llm, prompt=prompt_template)

C:\Users\Nagham\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [17]:
def extract_kg_elements(text):
    # Run the chain
    result = chain.run(text=text)
    
    # Parse the output
    parsed_output = output_parser.parse(result)
    
    return parsed_output.entities, parsed_output.relationships

In [18]:
sample_text = """
    Investigative people are inquisitive, analytical and like to learn. Investigative professions require one to carefully analyse the tasks at hand. Many investigative professionals are an expert in their particular field and solve problems through research and analysis. Typical hobbies of investigative people include intellectual sports such as Sudoku and chess, learning different languages or following additional courses just for the fun of it. One will encounter many investigative professions in academic education, IT, healthcare and innovative companies. Keywords: analytical, inquisitive, gaining knowledge, continuous learning, specialist.
"""

In [19]:
entities, relationships = extract_kg_elements(sample_text)

C:\Users\Nagham\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


In [20]:
print("Entities:")
for entity in entities:
    print(f"- {entity.name} ({entity.type})")

Entities:
- Investigative people (PersonalityTrait)
- Investigative professions (Profession)
- Sudoku (Intellectual sport)
- Chess (Intellectual sport)
- Languages (Knowledge)
- Academic education (Field of study)
- IT (Field of study)
- Healthcare (Field of study)
- Innovative companies (Field of study)
- Analytical (Ability)
- Inquisitive (PersonalityTrait)
- Continuous learning (Ability)
- Specialist (Profession)


In [21]:
print("\nRelationships:")
for rel in relationships:
    print(f"- {rel.subject} {rel.predicate} {rel.object}")


Relationships:
- Investigative people are characterized by Investigative professions
- Investigative professions require careful analysis
- Investigative people are experts in their particular field
- Investigative people solve problems through research and analysis
- Investigative people enjoy Sudoku
- Investigative people enjoy Chess
- Investigative people enjoy learning different languages
- Investigative people enjoy following additional courses
- Investigative professions can be found in academic education
- Investigative professions can be found in IT
- Investigative professions can be found in healthcare
- Investigative professions can be found in innovative companies
